In [ ]:
import threading as thr
import node as nd
import filechunk as fc
import numpy as np
import time

import matplotlib.pyplot as plt
%matplotlib inline

BASE_PORT = 30000
PATH = '/home/gee/Documents/Projetcs/aedii-bittorrent/music'
SONG = 'SalmonLikeTheFish - Zion.mp3'
X = list(range(2, 61, 1))

In [ ]:
nodes = []
threads = []

for i in range(X[-1]):
    nodes.append(nd.Node(('127.0.0.1', BASE_PORT+i)))
    threads.append(thr.Thread(target=nodes[i].listen))

y_1 = []
for i in range(len(X)):
    for j in range(X[i]):
        threads[j].start()
        while nodes[j].id == None:
            time.sleep(5e-3)
    
    for j in range(1, X[i]):
        nodes[j].enter_dht(nodes[0].addr)
    
    fc.add_file(PATH, SONG, [nodes[k].addr for k in range(X[i])])
    chunk_count = [len(node.dict) for node in nodes]
    y_1.append(np.std(chunk_count))
    
    for j in range(X[i]):
        nodes[j].alive = False
    for j in range(X[i]):
        threads[j].join()

    for j in range(X[i]):
        addr = nodes[j].addr
        nodes[j] = nd.Node(addr)
        threads[j] = thr.Thread(target=nodes[j].listen)

In [ ]:
import pandas as pd

df = pd.DataFrame()
df.index = X
df['Desvio Padrão'] = y_1

In [ ]:
nodes = []
threads = []

for i in range(X[-1]):
    nodes.append(nd.Node(('127.0.0.1', BASE_PORT+100+i)))
    threads.append(thr.Thread(target=nodes[i].listen))

y_3 = []
for i in range(len(X)):
    for j in range(X[i]):
        threads[j].start()
        while nodes[j].id == None:
            time.sleep(5e-3)
    
    for j in range(1, X[i]):
        nodes[j].enter_dht(nodes[0].addr)
    
    fc.add_file(PATH, SONG, [nodes[k].addr for k in range(X[i])])
    byte_count = [node.bytes_recv for node in nodes]
    y_3.append(np.sum(byte_count))
    
    for j in range(X[i]):
        nodes[j].alive = False
    for j in range(X[i]):
        threads[j].join()

    for j in range(X[i]):
        addr = nodes[j].addr
        nodes[j] = nd.Node(addr)
        threads[j] = thr.Thread(target=nodes[j].listen)

In [ ]:
df['Transferência Total'] = y_3
df.to_csv('~/Desktop/exp-1-3.csv', encoding='utf-8')

In [ ]:
X_CHUNK = list(range(1 << 12, (1 << 18) + 1, 1 << 12))

nodes = [nd.Node(('127.0.0.1', BASE_PORT+1000+i)) for i in range(60)]
threads = [thr.Thread(target=node.listen) for node in nodes]

y_2 = []
for chunk_size in X_CHUNK:
    fc.CHUNK_SIZE = chunk_size
    for i in range(len(threads)):
        threads[i].start()
        while nodes[i].id == None:
            time.sleep(5e-3)

    fc.add_file(PATH, SONG, [node.addr for node in nodes])
    chunk_count = [len(node.dict) for node in nodes]
    y_2.append(np.std(chunk_count))

    for node in nodes:
        node.alive = False
    for thread in threads:
        thread.join()

    for i in range(len(nodes)):
        addr = nodes[i].addr
        nodes[i] = nd.Node(addr)
        threads[i] = thr.Thread(target=nodes[i].listen)

In [ ]:
df_2 = pd.DataFrame()
df_2.index = X_CHUNK
df_2['Desvio Padrão'] = y_2

df_2.to_csv('~/Desktop/exp-2.csv', encoding='utf-8')